In [1]:
import cv2
import numpy as np
import pdb

from Managers import *

# Mog background example

In [2]:
mog = cv2.createBackgroundSubtractorMOG2()

In [3]:
def createMogBackground(image):
    return cv2.cvtColor(mog.apply(image), cv2.COLOR_GRAY2BGR)

In [4]:
cam = CameraManager('Mog Background', cv2.VideoCapture(0), createMogBackground).addCloseCallback()
cam.run()

# KNN background detection with cleanup

In [5]:
bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)

In [6]:
def createKnnBackground(image, l_thresh=244, k_height=3, k_width=3, min_size=1600):
    foreground = bs.apply(image)
    _, thresh = cv2.threshold(foreground, l_thresh, 255, cv2.THRESH_BINARY)
    
    ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k_height, k_width))
    dilate = cv2.dilate(thresh, ellipse, iterations=2)

    contours, hier = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        if cv2.contourArea(c) > min_size:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)
    
    return image

In [7]:
cam = CameraManager('Knn Background', cv2.VideoCapture(0), createKnnBackground).addCloseCallback()
cam.run()